In [1]:
import pandas as pd
import numpy as np
df_bandarban = pd.read_csv('../../1980-2024-dataset/bandarban_historical_weather_1980_2024.csv')
df_bandarban = df_bandarban.drop('district', axis =1 )

# Temperature

In [21]:
from sklearn.model_selection import train_test_split
import numpy as np

df = df_bandarban.copy()
if 'date' not in df.columns:
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])


# 'dew_point', 'atmospheric_pressure','max_temperature(degree C)' [collected based on the corrilation matrix]

# adding lagging for 3 days
lags = [1, 2, 3]
lag_cols = []

for lag in lags:
    df[f'dew_lag_{lag}'] = df['dew_point'].shift(lag)
    df[f'atm_lag_{lag}'] = df['atmospheric_pressure'].shift(lag)
    df[f'feels_lag_{lag}'] = df['max_temperature(degree C)'].shift(lag)

    lag_cols.extend([f'dew_lag_{lag}', f'atm_lag_{lag}', f'feels_lag_{lag}'])

# 7-day Rolling Average
df['dew_roll_7'] = df['dew_point'].transform(lambda x: x.rolling(window=7).mean())
df['atm_roll_7'] = df['atmospheric_pressure'].transform(lambda x: x.rolling(window=7).mean())
df['feels_roll_7'] = df['max_temperature(degree C)'].transform(lambda x: x.rolling(window =7).mean())


rolling_cols = ['dew_roll_7', 'atm_roll_7', 'feels_roll_7']

df = df.dropna().reset_index(drop=True)


df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df = df.sort_values('date').reset_index(drop=True)

df['day_of_year'] = df['date'].dt.dayofyear

def add_fourier_features(df,col,period,n_terms=10):
    for n in range(1, n_terms + 1):
        df[f'{col}_sin_{n}'] = np.sin(2 * np.pi * n * df.index / period)
        df[f'{col}_cos_{n}'] = np.cos(2 * np.pi * n * df.index / period)
    return df

df = add_fourier_features(df, 'day_of_year', period=365, n_terms=3)
fourier_cols = [c for c in df.columns if c.startswith('day_of_year_sin') or c.startswith('day_of_year_cos')]



In [ ]:
# FEATURES = [


#     ]

FEATURES = [ 'dew_point', 'atmospheric_pressure'] + fourier_cols # + lag_cols + rolling_cols # (first e eita uncomment kore feature importance dekhben. then 0.02 minmum importance gula note down kore prediction korben)
# 97.72/98.07 *-0.34

X = df[FEATURES]
y = df['temperature(degree C)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [23]:
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

    # Create an instance with specific parameters
early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=15,          # Wait 15 epochs for improvement before stopping
        restore_best_weights=True  # Very important: keeps the best version of your model
    )

    # 1. Scale the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

    # 2. Reshape for GRU: (samples, time_steps, features)
    # Here we use time_steps=1. If you want sequences, you'd need a sliding window function.
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

    # Split data (matching your non-shuffle 80/20 split)
split_idx = int(len(X_reshaped) * 0.8)
X_train, X_test = X_reshaped[:split_idx], X_reshaped[split_idx:]
y_train, y_test = y_scaled[:split_idx], y_scaled[split_idx:]


def build_gru(input_shape):
        model = Sequential([
            GRU(64, activation='tanh', input_shape=input_shape, return_sequences=False, recurrent_dropout=0.1),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(1) # Output layer for regression
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    # Train initial model
gru_model = build_gru((X_train.shape[1], X_train.shape[2]))
gru_model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        batch_size=64,
        validation_split=0.2,
        callbacks=[early_stopping], 
        verbose=1
    )

    # Predict and Inverse Scale
y_pred_scaled = gru_model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler_y.inverse_transform(y_test)

mse_gru = mean_squared_error(y_test_unscaled, y_pred)
rmse_gru = np.sqrt(mse_gru)
r2_gru = r2_score(y_test_unscaled, y_pred)

print(f"\nGRU Results for temperature :")
print(f'Mean Squared Error: {mse_gru:.4f}')
print(f'RMSE: {rmse_gru:.4f}')
print(f'R² Score: {r2_gru:.4f}')

tscv = TimeSeriesSplit(n_splits=5)
rmse_list_gru = []
r2_list_gru = []
mse_list_gru = []

for train_index, test_index in tscv.split(X_reshaped):
        X_train_kf, X_test_kf = X_reshaped[train_index], X_reshaped[test_index]
        y_train_kf, y_test_kf = y_scaled[train_index], y_scaled[test_index]

        # Rebuild/Reset model for each fold
        gru_kf = build_gru((X_train_kf.shape[1], X_train_kf.shape[2]))
        gru_kf.fit(X_train_kf, y_train_kf, epochs=30, batch_size=32, verbose=0)

        # Predict and Inverse
        y_pred_kf_scaled = gru_kf.predict(X_test_kf)
        y_pred_kf = scaler_y.inverse_transform(y_pred_kf_scaled)
        y_test_kf_unscaled = scaler_y.inverse_transform(y_test_kf)

        mse_kf = mean_squared_error(y_test_kf_unscaled, y_pred_kf)
        rmse_list_gru.append(np.sqrt(mse_kf))
        mse_list_gru.append(mse_kf)
        r2_list_gru.append(r2_score(y_test_kf_unscaled, y_pred_kf))

average_r2_gru = np.mean(r2_list_gru)
average_mse_gru = np.mean(mse_list_gru)
average_rmse_gru = np.mean(rmse_list_gru)

print("\n")
print(f"Average RMSE from CV: {average_rmse_gru:.4f}")
print(f"Average R² from CV: {average_r2_gru:.4f}")
print(f"Average MSE: {average_mse_gru:.4f}")
print(f"Individual Fold RMSEs: {rmse_list_gru}")

diff = (r2_gru - np.mean(r2_list_gru))*100
print ( f'\n R2 ~ {diff:.4f}')

Epoch 1/100


c:\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.4531 - val_loss: 0.1298
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1418 - val_loss: 0.1210
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1267 - val_loss: 0.1091
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1230 - val_loss: 0.1099
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1166 - val_loss: 0.1055
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1112 - val_loss: 0.1038
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1072 - val_loss: 0.1064
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1075 - val_loss: 0.1028
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1056 - val_loss: 0.1011
Epoch 10/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1037 - val_loss: 0.1065
Epoch 11/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1049 - val_loss: 0.1044
Epoch 12/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1034 - val_l

c:\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


...

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


c:\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


c:\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


Average RMSE from CV: 1.1783
Average R² from CV: 0.8843
Average MSE: 1.3962
Individual Fold RMSEs: [np.float64(1.233016720645276), np.float64(1.077586824255452), np.float64(1.1697958496604595), np.float64(1.096276699263201), np.float64(1.314979486571461)]

 R2 ~ -2.5419


In [24]:
# Permutation Importance Implementation

import pandas as pd
import numpy as np

def calculate_permutation_importance(model, X_val, y_val, scaler_y, feature_names):
    """
    Calculates importance by measuring how much the MSE increases 
    when a single feature is randomly shuffled.
    """
    # Baseline prediction
    baseline_preds = model.predict(X_val, verbose=0)
    baseline_mse = mean_squared_error(scaler_y.inverse_transform(y_val), 
                                     scaler_y.inverse_transform(baseline_preds))
    
    importances = []
    
    for i in range(X_val.shape[2]):  # Iterate through each feature
        save = X_val[:, :, i].copy()
        
        # Shuffle the current feature across all samples
        np.random.shuffle(X_val[:, :, i])
        
        # Predict with shuffled feature
        shuffled_preds = model.predict(X_val, verbose=0)
        shuffled_mse = mean_squared_error(scaler_y.inverse_transform(y_val), 
                                         scaler_y.inverse_transform(shuffled_preds))
        
        # Importance is the increase in error
        importances.append(max(0, shuffled_mse - baseline_mse))
        
        # Restore the original feature values
        X_val[:, :, i] = save

    # Create DataFrame
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    return importance_df.sort_values(by='Importance', ascending=False)

# --- Execute ---
# Note: Use your X_test and y_test from the previous step
feature_importance_gru = calculate_permutation_importance(
    gru_model, 
    X_test, 
    y_test, 
    scaler_y, 
    FEATURES
)


print("\nGRU Permutation Feature Importances:")
print(feature_importance_gru)


GRU Permutation Feature Importances:
                Feature  Importance
0             dew_point    9.641436
4     day_of_year_sin_2    0.676416
5     day_of_year_cos_2    0.659937
3     day_of_year_cos_1    0.615340
6     day_of_year_sin_3    0.433408
7     day_of_year_cos_3    0.270914
1  atmospheric_pressure    0.161488
2     day_of_year_sin_1    0.036409


## predicitng temperature

In [26]:
# generate a dataframe for predicted tempeature for 2025 and 2026 
future_dates = pd.date_range(start='2025-01-01', end='2026-12-31', freq='D')

future_df = pd.DataFrame({'date': future_dates})

future_df['year'] = future_df['date'].dt.year
future_df['month'] = future_df['date'].dt.month
future_df['day'] = future_df['date'].dt.day
future_df['day_of_year'] = future_df['date'].dt.dayofyear

future_df = add_fourier_features(future_df, 'day_of_year', period=365, n_terms=3)
# For simplicity, we'll fill lag and rolling features with the mean of the training data

for lag in lags:
    future_df[f'dew_lag_{lag}'] = df[f'dew_lag_{lag}'].mean()
    future_df[f'atm_lag_{lag}'] = df[f'atm_lag_{lag}'].mean()
    future_df[f'feels_lag_{lag}'] = df[f'feels_lag_{lag}'].mean()


future_df['dew_roll_7'] = df['dew_roll_7'].mean()
future_df['atm_roll_7'] = df['atm_roll_7'].mean()
future_df['feels_roll_7'] = df['feels_roll_7'].mean()

FEATURES = [ 'dew_point', 'atmospheric_pressure'] + fourier_cols
# Prepare features for prediction
future_X = future_df[FEATURES]
# Scale features
future_X_scaled = scaler_X.transform(future_X)
# Reshape for GRU
future_X_reshaped = future_X_scaled.reshape((future_X_scaled.shape[0], 1, future_X_scaled.shape[1]))
# Predict future temperatures
future_preds_scaled = gru_model.predict(future_X_reshaped, verbose=0)
future_preds = scaler_y.inverse_transform(future_preds_scaled)
future_df['predicted_temperature'] = future_preds

print("\nPredicted Temperatures for 2025-2026:")
print(future_df[['date', 'predicted_temperature']])






KeyError: "['dew_point', 'atmospheric_pressure'] not in index"

In [19]:
# For 2026, repeat the same process (assuming you have a 2026 dataframe)

# Prepare 2026 data
df_2026 = df_bandarban.copy()
if 'date' not in df_2026.columns:
    df_2026['date'] = pd.to_datetime(df_2026[['year', 'month', 'day']])

# Apply lag and rolling features (same as training)
for lag in [1, 2, 3]:
    df_2026[f'dew_lag_{lag}'] = df_2026['dew_point'].shift(lag)
    df_2026[f'atm_lag_{lag}'] = df_2026['atmospheric_pressure'].shift(lag)
    df_2026[f'feels_lag_{lag}'] = df_2026['max_temperature(degree C)'].shift(lag)

df_2026['dew_roll_7'] = df_2026['dew_point'].transform(lambda x: x.rolling(window=7).mean())
df_2026['atm_roll_7'] = df_2026['atmospheric_pressure'].transform(lambda x: x.rolling(window=7).mean())
df_2026['feels_roll_7'] = df_2026['max_temperature(degree C)'].transform(lambda x: x.rolling(window=7).mean())

df_2026['day_of_year'] = df_2026['date'].dt.dayofyear
df_2026 = add_fourier_features(df_2026, 'day_of_year', period=365, n_terms=3)

df_2026 = df_2026.dropna().reset_index(drop=True)

# Prepare features for prediction
X_2026 = df_2026[FEATURES]
X_2026_scaled = scaler_X.transform(X_2026)
X_2026_reshaped = X_2026_scaled.reshape((X_2026_scaled.shape[0], 1, X_2026_scaled.shape[1]))

# Predict
y_2026_pred_scaled = gru_model.predict(X_2026_reshaped)
y_2026_pred = scaler_y.inverse_transform(y_2026_pred_scaled)

df_pred_2026 = df_2026[['date']].copy()
df_pred_2026['predicted_temperature'] = y_2026_pred.flatten()

df_pred_2026.tail()

127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,date,predicted_temperature
4033,2024-12-27,21.335312
4034,2024-12-28,21.490690
4035,2024-12-29,20.723204
4036,2024-12-30,20.578735
4037,2024-12-31,20.703154


In [8]:
df_bandarban_2025 = pd.read_csv('../../2025-dataset/bandarban_historical_weather_2025.csv')
# df_bandarban_2025 = df_bandarban.drop('district', axis =1 )

In [ ]:
df_bandarban_2025.head()

,district,day,month,year,temperature(degree C),feels_like(degree C),max_temperature(degree C),minimum_temperature(degree C),humidity,precipitation,windspeed,atmospheric_pressure,UV,solar_radiation,dew_point
0,Bandarban,1,1,2025,19.3,19.3,25.8,14.7,82.9,0.0,18.4,1013.6,7.0,194.4,16.2
1,Bandarban,2,1,2025,18.6,18.6,23.3,15.8,86.5,0.0,16.6,1014.1,7.0,193.8,16.1
2,Bandarban,3,1,2025,18.3,18.3,22.6,15.3,90.9,0.0,12.1,1014.8,7.0,172.7,16.7
3,Bandarban,4,1,2025,18.8,18.8,23.4,15.0,88.9,0.0,11.2,1013.7,7.0,172.9,16.9
4,Bandarban,5,1,2025,20.7,20.9,27.5,15.4,83.4,0.0,18.4,1014.6,7.0,169.1,17.4
